In [40]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from bokeh.core.properties import value
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, HoverTool, LabelSet, LinearColorMapper, Panel, Select
from bokeh.palettes import Spectral6, Spectral11, Viridis256
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.transform import factor_cmap, transform

In [41]:
import chardet
# Détecter l'encodage
with open("avito_car_dataset_ALL.csv", 'rb') as f:
    result = chardet.detect(f.read())
# Lire le fichier avec l'encodage détecté
df = pd.read_csv("avito_car_dataset_ALL.csv", encoding=result['encoding'])
# probleme de lecture en encodage utf-8, c'est pour cela on a detecter d'abord l'encodage du fichier, puis on a lu ce fichier avec cet encodage

<h2>Part1: Data Cleaning & Preparation</h2>

In [5]:
df.head()

,Unnamed: 0,Lien,Ville,Secteur,Marque,Modèle,Année-Modèle,Kilométrage,Type de carburant,Puissance fiscale,...,Caméra de recul,Vitres électriques,ABS,ESP,Régulateur de vitesse,Limiteur de vitesse,CD/MP3/Bluetooth,Ordinateur de bord,Verrouillage centralisé à distance,Prix
0,0,https://www.avito.ma/fr/massira_2/voitures/FIA...,Temara,Massira 2,Fiat,Punto,2007,200 000 - 249 999,Diesel,5,...,False,False,True,False,False,False,True,False,False,60000
1,1,https://www.avito.ma/fr/temara/voitures/Dacia_...,Temara,NaN,Dacia,Dokker Van,2013,400 000 - 449 999,Diesel,6,...,False,False,False,False,False,False,False,False,False,70000
2,2,https://www.avito.ma/fr/casablanca/voitures/Da...,Casablanca,NaN,Dacia,Dokker,2014,160 000 - 169 999,Diesel,6,...,False,False,False,False,False,False,False,False,False,90000
3,3,https://www.avito.ma/fr/casablanca/voitures/to...,Casablanca,NaN,Volkswagen,Touareg,2005,0 - 4 999,Diesel,10,...,False,False,False,False,False,False,False,False,False,90000
4,4,https://www.avito.ma/fr/dakhla/voitures/Toyota...,Dakhla,NaN,Toyota,Prado,2007,200 000 - 249 999,Diesel,12,...,False,False,True,False,False,False,True,False,False,97000


In [6]:
df.columns

Index(['Unnamed: 0', 'Lien', 'Ville', 'Secteur', 'Marque', 'Modèle',
       'Année-Modèle', 'Kilométrage', 'Type de carburant', 'Puissance fiscale',
       'Boite de vitesses', 'Nombre de portes', 'Origine', 'Première main',
       'État', 'Jantes aluminium', 'Airbags', 'Climatisation',
       'Système de navigation/GPS', 'Toit ouvrant', 'Sièges cuir',
       'Radar de recul', 'Caméra de recul', 'Vitres électriques', 'ABS', 'ESP',
       'Régulateur de vitesse', 'Limiteur de vitesse', 'CD/MP3/Bluetooth',
       'Ordinateur de bord', 'Verrouillage centralisé à distance', 'Prix'],
      dtype='object')

In [7]:
df.drop(columns=['Secteur','Lien'], inplace = True)

In [8]:
#si on utilise df.info(), il affiche seulement les infos d'une partie des colonnes, donc on afficher chaque 16 colonnes pour qu'on peut voir les infos de toutes les colonnes
df.iloc[:, :16].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24776 entries, 0 to 24775
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         24776 non-null  int64  
 1   Ville              24776 non-null  object 
 2   Marque             24776 non-null  object 
 3   Modèle             24776 non-null  object 
 4   Année-Modèle       24776 non-null  int64  
 5   Kilométrage        24776 non-null  object 
 6   Type de carburant  24776 non-null  object 
 7   Puissance fiscale  24776 non-null  int64  
 8   Boite de vitesses  24776 non-null  object 
 9   Nombre de portes   20396 non-null  float64
 10  Origine            19489 non-null  object 
 11  Première main      18567 non-null  object 
 12  État               18098 non-null  object 
 13  Jantes aluminium   24776 non-null  bool   
 14  Airbags            24776 non-null  bool   
 15  Climatisation      24776 non-null  bool   
dtypes: bool(3), float64(1)

In [9]:
df.iloc[:, -16:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24776 entries, 0 to 24775
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   Airbags                             24776 non-null  bool 
 1   Climatisation                       24776 non-null  bool 
 2   Système de navigation/GPS           24776 non-null  bool 
 3   Toit ouvrant                        24776 non-null  bool 
 4   Sièges cuir                         24776 non-null  bool 
 5   Radar de recul                      24776 non-null  bool 
 6   Caméra de recul                     24776 non-null  bool 
 7   Vitres électriques                  24776 non-null  bool 
 8   ABS                                 24776 non-null  bool 
 9   ESP                                 24776 non-null  bool 
 10  Régulateur de vitesse               24776 non-null  bool 
 11  Limiteur de vitesse                 24776 non-null  bool 
 12  CD/M

In [10]:
df=df.rename(columns={'Unnamed: 0': 'car_id'})
df

,car_id,Ville,Marque,Modèle,Année-Modèle,Kilométrage,Type de carburant,Puissance fiscale,Boite de vitesses,Nombre de portes,...,Caméra de recul,Vitres électriques,ABS,ESP,Régulateur de vitesse,Limiteur de vitesse,CD/MP3/Bluetooth,Ordinateur de bord,Verrouillage centralisé à distance,Prix
0,0,Temara,Fiat,Punto,2007,200 000 - 249 999,Diesel,5,Manuelle,5.0,...,False,False,True,False,False,False,True,False,False,60000
1,1,Temara,Dacia,Dokker Van,2013,400 000 - 449 999,Diesel,6,Manuelle,3.0,...,False,False,False,False,False,False,False,False,False,70000
2,2,Casablanca,Dacia,Dokker,2014,160 000 - 169 999,Diesel,6,Manuelle,5.0,...,False,False,False,False,False,False,False,False,False,90000
3,3,Casablanca,Volkswagen,Touareg,2005,0 - 4 999,Diesel,10,Automatique,5.0,...,False,False,False,False,False,False,False,False,False,90000
4,4,Dakhla,Toyota,Prado,2007,200 000 - 249 999,Diesel,12,Manuelle,5.0,...,False,False,True,False,False,False,True,False,False,97000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24771,24771,Marrakech,mini,cabrio,2011,70 000 - 74 999,Essence,9,Manuelle,3.0,...,False,False,True,True,False,False,False,False,False,180000
24772,24772,Safi,mini,cooper,2007,190 000 - 199 999,Essence,9,Manuelle,3.0,...,False,False,True,True,False,False,True,False,False,63000
24773,24773,Salé,mini,one,2006,10 000 - 14 999,Essence,8,Manuelle,3.0,...,False,False,True,False,False,False,True,False,False,60000
24774,24774,Casablanca,mini,cooper,2010,140 000 - 149 999,Essence,9,Automatique,3.0,...,False,False,True,True,False,False,True,False,False,85000


In [11]:
#suppression de Caméra de recul, c'est la meme chose que Radar de recul:
df.drop(columns=['Caméra de recul'], inplace=True)

In [12]:
df

,car_id,Ville,Marque,Modèle,Année-Modèle,Kilométrage,Type de carburant,Puissance fiscale,Boite de vitesses,Nombre de portes,...,Radar de recul,Vitres électriques,ABS,ESP,Régulateur de vitesse,Limiteur de vitesse,CD/MP3/Bluetooth,Ordinateur de bord,Verrouillage centralisé à distance,Prix
0,0,Temara,Fiat,Punto,2007,200 000 - 249 999,Diesel,5,Manuelle,5.0,...,False,False,True,False,False,False,True,False,False,60000
1,1,Temara,Dacia,Dokker Van,2013,400 000 - 449 999,Diesel,6,Manuelle,3.0,...,False,False,False,False,False,False,False,False,False,70000
2,2,Casablanca,Dacia,Dokker,2014,160 000 - 169 999,Diesel,6,Manuelle,5.0,...,False,False,False,False,False,False,False,False,False,90000
3,3,Casablanca,Volkswagen,Touareg,2005,0 - 4 999,Diesel,10,Automatique,5.0,...,False,False,False,False,False,False,False,False,False,90000
4,4,Dakhla,Toyota,Prado,2007,200 000 - 249 999,Diesel,12,Manuelle,5.0,...,False,False,True,False,False,False,True,False,False,97000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24771,24771,Marrakech,mini,cabrio,2011,70 000 - 74 999,Essence,9,Manuelle,3.0,...,False,False,True,True,False,False,False,False,False,180000
24772,24772,Safi,mini,cooper,2007,190 000 - 199 999,Essence,9,Manuelle,3.0,...,False,False,True,True,False,False,True,False,False,63000
24773,24773,Salé,mini,one,2006,10 000 - 14 999,Essence,8,Manuelle,3.0,...,False,False,True,False,False,False,True,False,False,60000
24774,24774,Casablanca,mini,cooper,2010,140 000 - 149 999,Essence,9,Automatique,3.0,...,False,False,True,True,False,False,True,False,False,85000


In [13]:
def process_kilom(df):
    # Remplacer les espaces par le vide ''
    df['Kilométrage'] = df['Kilométrage'].str.replace(' ', '')
    # Créer deux colonnes "main1" et "main2" avec des valeurs NaN
    df['main1'] = np.nan
    df['main2'] = np.nan
    # Parcourir chaque valeur de la colonne "Kilométrage" pour extraire le 1er Kilométrage et 2eme.
    for i, value in enumerate(df['Kilométrage']):
        parts = value.split('-')
        m1 = int(parts[0].strip()) if parts else 0
        m2 = int(parts[1].strip()) if len(parts) > 1 else 0
        if m1 == 0:  # Si m1 est égal à 0
            df.at[i, 'main1'] = m2
            df.at[i, 'main2'] = m1
        else:  # Sinon
            df.at[i, 'main1'] = m1
            df.at[i, 'main2'] = m2
    return df
df = process_kilom(df)
df

,car_id,Ville,Marque,Modèle,Année-Modèle,Kilométrage,Type de carburant,Puissance fiscale,Boite de vitesses,Nombre de portes,...,ABS,ESP,Régulateur de vitesse,Limiteur de vitesse,CD/MP3/Bluetooth,Ordinateur de bord,Verrouillage centralisé à distance,Prix,main1,main2
0,0,Temara,Fiat,Punto,2007,200000-249999,Diesel,5,Manuelle,5.0,...,True,False,False,False,True,False,False,60000,200000.0,249999.0
1,1,Temara,Dacia,Dokker Van,2013,400000-449999,Diesel,6,Manuelle,3.0,...,False,False,False,False,False,False,False,70000,400000.0,449999.0
2,2,Casablanca,Dacia,Dokker,2014,160000-169999,Diesel,6,Manuelle,5.0,...,False,False,False,False,False,False,False,90000,160000.0,169999.0
3,3,Casablanca,Volkswagen,Touareg,2005,0-4999,Diesel,10,Automatique,5.0,...,False,False,False,False,False,False,False,90000,4999.0,0.0
4,4,Dakhla,Toyota,Prado,2007,200000-249999,Diesel,12,Manuelle,5.0,...,True,False,False,False,True,False,False,97000,200000.0,249999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24771,24771,Marrakech,mini,cabrio,2011,70000-74999,Essence,9,Manuelle,3.0,...,True,True,False,False,False,False,False,180000,70000.0,74999.0
24772,24772,Safi,mini,cooper,2007,190000-199999,Essence,9,Manuelle,3.0,...,True,True,False,False,True,False,False,63000,190000.0,199999.0
24773,24773,Salé,mini,one,2006,10000-14999,Essence,8,Manuelle,3.0,...,True,False,False,False,True,False,False,60000,10000.0,14999.0
24774,24774,Casablanca,mini,cooper,2010,140000-149999,Essence,9,Automatique,3.0,...,True,True,False,False,True,False,False,85000,140000.0,149999.0


In [14]:
#mettre main1 et main2 a cote de Kilometrage:
cols_to_move = ['main1', 'main2'] # Colonnes à déplacer
after_col = 'Kilométrage' # Colonne après laquelle déplacer les colonnes
list_of_cols = df.columns.tolist() # Nouvelle liste d'ordre des colonnes

for col in cols_to_move: #suppression des colonnes a ordonner de la liste des colonnes
    list_of_cols.remove(col)
    
insert_idx = list_of_cols.index(after_col) + 1

for col in cols_to_move: #boucle pour réinsertion des deux colonnes
    list_of_cols.insert(insert_idx, col)
    insert_idx += 1

df = df[list_of_cols] # Réindexer les colonnes
df

,car_id,Ville,Marque,Modèle,Année-Modèle,Kilométrage,main1,main2,Type de carburant,Puissance fiscale,...,Radar de recul,Vitres électriques,ABS,ESP,Régulateur de vitesse,Limiteur de vitesse,CD/MP3/Bluetooth,Ordinateur de bord,Verrouillage centralisé à distance,Prix
0,0,Temara,Fiat,Punto,2007,200000-249999,200000.0,249999.0,Diesel,5,...,False,False,True,False,False,False,True,False,False,60000
1,1,Temara,Dacia,Dokker Van,2013,400000-449999,400000.0,449999.0,Diesel,6,...,False,False,False,False,False,False,False,False,False,70000
2,2,Casablanca,Dacia,Dokker,2014,160000-169999,160000.0,169999.0,Diesel,6,...,False,False,False,False,False,False,False,False,False,90000
3,3,Casablanca,Volkswagen,Touareg,2005,0-4999,4999.0,0.0,Diesel,10,...,False,False,False,False,False,False,False,False,False,90000
4,4,Dakhla,Toyota,Prado,2007,200000-249999,200000.0,249999.0,Diesel,12,...,False,False,True,False,False,False,True,False,False,97000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24771,24771,Marrakech,mini,cabrio,2011,70000-74999,70000.0,74999.0,Essence,9,...,False,False,True,True,False,False,False,False,False,180000
24772,24772,Safi,mini,cooper,2007,190000-199999,190000.0,199999.0,Essence,9,...,False,False,True,True,False,False,True,False,False,63000
24773,24773,Salé,mini,one,2006,10000-14999,10000.0,14999.0,Essence,8,...,False,False,True,False,False,False,True,False,False,60000
24774,24774,Casablanca,mini,cooper,2010,140000-149999,140000.0,149999.0,Essence,9,...,False,False,True,True,False,False,True,False,False,85000


In [15]:
df['main1'] = df['main1'].astype('int64')
df['main2'] = df['main2'].astype('int64')

In [16]:
df.iloc[:, :16].isnull().sum()

car_id                  0
Ville                   0
Marque                  0
Modèle                  0
Année-Modèle            0
Kilométrage             0
main1                   0
main2                   0
Type de carburant       0
Puissance fiscale       0
Boite de vitesses       0
Nombre de portes     4380
Origine              5287
Première main        6209
État                 6678
Jantes aluminium        0
dtype: int64

In [17]:
df.iloc[:, -16:].isnull().sum()

Jantes aluminium                      0
Airbags                               0
Climatisation                         0
Système de navigation/GPS             0
Toit ouvrant                          0
Sièges cuir                           0
Radar de recul                        0
Vitres électriques                    0
ABS                                   0
ESP                                   0
Régulateur de vitesse                 0
Limiteur de vitesse                   0
CD/MP3/Bluetooth                      0
Ordinateur de bord                    0
Verrouillage centralisé à distance    0
Prix                                  0
dtype: int64

<h5>On remplie les valeurs manquantes des colonnes suivantes: Première main, Nombre de portes, État et Origine
       - Première main: on va mettre True si main2=0, False sinon. car il ya des cas ou on a True alors que le deuxiem kilometrage est non nulle(ligne 24771 par exemple)
       - Etat: on remplie les cases vides selon les estimations suivantes:
              Excellente: 1er main + main1 moins de 20,000 km
              Tres Bon: Entre 20,000 km et 50,000 km
              Bon: Plus de 50,000 km
       - Nombre de porte: on copie le meme nombre de porte que les autres ayant le meme Modele et la meme marque.
       - Origine: pour les cas vides NaN, on va mettre 'Non spécifié'. </h5>

In [18]:
#changement de type de la colonne Premiere main de Object en booleen:
df['Première main'] = df['Première main'].astype(bool)
#remplissage de Etat:
df['Première main'] = df['main2'] == 0
df

,car_id,Ville,Marque,Modèle,Année-Modèle,Kilométrage,main1,main2,Type de carburant,Puissance fiscale,...,Radar de recul,Vitres électriques,ABS,ESP,Régulateur de vitesse,Limiteur de vitesse,CD/MP3/Bluetooth,Ordinateur de bord,Verrouillage centralisé à distance,Prix
0,0,Temara,Fiat,Punto,2007,200000-249999,200000,249999,Diesel,5,...,False,False,True,False,False,False,True,False,False,60000
1,1,Temara,Dacia,Dokker Van,2013,400000-449999,400000,449999,Diesel,6,...,False,False,False,False,False,False,False,False,False,70000
2,2,Casablanca,Dacia,Dokker,2014,160000-169999,160000,169999,Diesel,6,...,False,False,False,False,False,False,False,False,False,90000
3,3,Casablanca,Volkswagen,Touareg,2005,0-4999,4999,0,Diesel,10,...,False,False,False,False,False,False,False,False,False,90000
4,4,Dakhla,Toyota,Prado,2007,200000-249999,200000,249999,Diesel,12,...,False,False,True,False,False,False,True,False,False,97000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24771,24771,Marrakech,mini,cabrio,2011,70000-74999,70000,74999,Essence,9,...,False,False,True,True,False,False,False,False,False,180000
24772,24772,Safi,mini,cooper,2007,190000-199999,190000,199999,Essence,9,...,False,False,True,True,False,False,True,False,False,63000
24773,24773,Salé,mini,one,2006,10000-14999,10000,14999,Essence,8,...,False,False,True,False,False,False,True,False,False,60000
24774,24774,Casablanca,mini,cooper,2010,140000-149999,140000,149999,Essence,9,...,False,False,True,True,False,False,True,False,False,85000


In [19]:
df['Première main'].isnull().sum()

0

In [20]:
# Remplissage de État:
for i in range(len(df)):
    if pd.isna(df.at[i, 'État']): #verifie les cases vide(NaN) de la colonne État
        if df.at[i, 'Première main'] and df.at[i, 'main1'] < 20000:
            df.at[i, 'État'] = 'Excellente'
        if not df.at[i, 'Première main'] and df.at[i, 'main1'] < 20000:
            df.at[i, 'État'] = 'Très Bon'
        elif 20000 <= df.at[i, 'main2'] <= 50000:
            df.at[i, 'État'] = 'Très Bon'
        elif df.at[i, 'main2'] > 50000:
            df.at[i, 'État'] = 'Bon'
df

,car_id,Ville,Marque,Modèle,Année-Modèle,Kilométrage,main1,main2,Type de carburant,Puissance fiscale,...,Radar de recul,Vitres électriques,ABS,ESP,Régulateur de vitesse,Limiteur de vitesse,CD/MP3/Bluetooth,Ordinateur de bord,Verrouillage centralisé à distance,Prix
0,0,Temara,Fiat,Punto,2007,200000-249999,200000,249999,Diesel,5,...,False,False,True,False,False,False,True,False,False,60000
1,1,Temara,Dacia,Dokker Van,2013,400000-449999,400000,449999,Diesel,6,...,False,False,False,False,False,False,False,False,False,70000
2,2,Casablanca,Dacia,Dokker,2014,160000-169999,160000,169999,Diesel,6,...,False,False,False,False,False,False,False,False,False,90000
3,3,Casablanca,Volkswagen,Touareg,2005,0-4999,4999,0,Diesel,10,...,False,False,False,False,False,False,False,False,False,90000
4,4,Dakhla,Toyota,Prado,2007,200000-249999,200000,249999,Diesel,12,...,False,False,True,False,False,False,True,False,False,97000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24771,24771,Marrakech,mini,cabrio,2011,70000-74999,70000,74999,Essence,9,...,False,False,True,True,False,False,False,False,False,180000
24772,24772,Safi,mini,cooper,2007,190000-199999,190000,199999,Essence,9,...,False,False,True,True,False,False,True,False,False,63000
24773,24773,Salé,mini,one,2006,10000-14999,10000,14999,Essence,8,...,False,False,True,False,False,False,True,False,False,60000
24774,24774,Casablanca,mini,cooper,2010,140000-149999,140000,149999,Essence,9,...,False,False,True,True,False,False,True,False,False,85000


In [21]:
df['État'].isnull().sum()

0

In [22]:
#Remplissage Nombre de portes:
for i, row in df.iterrows():
    if pd.isna(row['Nombre de portes']):
        # selectionner les lignes avec le meme Modèle, Marque et Année-Modèle
        same_car = df[
            (df['Marque'] == row['Marque']) &
            (df['Modèle'] == row['Modèle']) &
            df['Nombre de portes'].notna()
        ]
        # Si on trouve une meme voiture, remplir dans la valeur manquante, le meme nombre de portes
        if not same_car.empty:
            df.at[i, 'Nombre de portes'] = same_car['Nombre de portes'].iloc[0]

df

,car_id,Ville,Marque,Modèle,Année-Modèle,Kilométrage,main1,main2,Type de carburant,Puissance fiscale,...,Radar de recul,Vitres électriques,ABS,ESP,Régulateur de vitesse,Limiteur de vitesse,CD/MP3/Bluetooth,Ordinateur de bord,Verrouillage centralisé à distance,Prix
0,0,Temara,Fiat,Punto,2007,200000-249999,200000,249999,Diesel,5,...,False,False,True,False,False,False,True,False,False,60000
1,1,Temara,Dacia,Dokker Van,2013,400000-449999,400000,449999,Diesel,6,...,False,False,False,False,False,False,False,False,False,70000
2,2,Casablanca,Dacia,Dokker,2014,160000-169999,160000,169999,Diesel,6,...,False,False,False,False,False,False,False,False,False,90000
3,3,Casablanca,Volkswagen,Touareg,2005,0-4999,4999,0,Diesel,10,...,False,False,False,False,False,False,False,False,False,90000
4,4,Dakhla,Toyota,Prado,2007,200000-249999,200000,249999,Diesel,12,...,False,False,True,False,False,False,True,False,False,97000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24771,24771,Marrakech,mini,cabrio,2011,70000-74999,70000,74999,Essence,9,...,False,False,True,True,False,False,False,False,False,180000
24772,24772,Safi,mini,cooper,2007,190000-199999,190000,199999,Essence,9,...,False,False,True,True,False,False,True,False,False,63000
24773,24773,Salé,mini,one,2006,10000-14999,10000,14999,Essence,8,...,False,False,True,False,False,False,True,False,False,60000
24774,24774,Casablanca,mini,cooper,2010,140000-149999,140000,149999,Essence,9,...,False,False,True,True,False,False,True,False,False,85000


In [23]:
df['Nombre de portes'].isnull().sum()

39

In [24]:
#remplacer NaN dans Origine par Non Spécifié':
df['Origine'].fillna('Non Spécifié', inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_2788\4191175740.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Origine'].fillna('Non Spécifié', inplace=True)


39 Modele de voiture avec nombre de portes inconues.

In [25]:
df.iloc[:, :16].isnull().sum()

car_id                0
Ville                 0
Marque                0
Modèle                0
Année-Modèle          0
Kilométrage           0
main1                 0
main2                 0
Type de carburant     0
Puissance fiscale     0
Boite de vitesses     0
Nombre de portes     39
Origine               0
Première main         0
État                  0
Jantes aluminium      0
dtype: int64

In [26]:
df.iloc[:, -16:].isnull().sum()

Jantes aluminium                      0
Airbags                               0
Climatisation                         0
Système de navigation/GPS             0
Toit ouvrant                          0
Sièges cuir                           0
Radar de recul                        0
Vitres électriques                    0
ABS                                   0
ESP                                   0
Régulateur de vitesse                 0
Limiteur de vitesse                   0
CD/MP3/Bluetooth                      0
Ordinateur de bord                    0
Verrouillage centralisé à distance    0
Prix                                  0
dtype: int64

In [27]:
df.drop_duplicates()

,car_id,Ville,Marque,Modèle,Année-Modèle,Kilométrage,main1,main2,Type de carburant,Puissance fiscale,...,Radar de recul,Vitres électriques,ABS,ESP,Régulateur de vitesse,Limiteur de vitesse,CD/MP3/Bluetooth,Ordinateur de bord,Verrouillage centralisé à distance,Prix
0,0,Temara,Fiat,Punto,2007,200000-249999,200000,249999,Diesel,5,...,False,False,True,False,False,False,True,False,False,60000
1,1,Temara,Dacia,Dokker Van,2013,400000-449999,400000,449999,Diesel,6,...,False,False,False,False,False,False,False,False,False,70000
2,2,Casablanca,Dacia,Dokker,2014,160000-169999,160000,169999,Diesel,6,...,False,False,False,False,False,False,False,False,False,90000
3,3,Casablanca,Volkswagen,Touareg,2005,0-4999,4999,0,Diesel,10,...,False,False,False,False,False,False,False,False,False,90000
4,4,Dakhla,Toyota,Prado,2007,200000-249999,200000,249999,Diesel,12,...,False,False,True,False,False,False,True,False,False,97000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24771,24771,Marrakech,mini,cabrio,2011,70000-74999,70000,74999,Essence,9,...,False,False,True,True,False,False,False,False,False,180000
24772,24772,Safi,mini,cooper,2007,190000-199999,190000,199999,Essence,9,...,False,False,True,True,False,False,True,False,False,63000
24773,24773,Salé,mini,one,2006,10000-14999,10000,14999,Essence,8,...,False,False,True,False,False,False,True,False,False,60000
24774,24774,Casablanca,mini,cooper,2010,140000-149999,140000,149999,Essence,9,...,False,False,True,True,False,False,True,False,False,85000


In [28]:
def calculer_kilometrage(row):
    if row['Première main']:
        return row['main1']
    else:
        return row['main2']
df['Kilometrage'] = df.apply(calculer_kilometrage, axis=1)

In [29]:
df.drop(columns='Kilométrage', inplace=True)

In [30]:
df

,car_id,Ville,Marque,Modèle,Année-Modèle,main1,main2,Type de carburant,Puissance fiscale,Boite de vitesses,...,Vitres électriques,ABS,ESP,Régulateur de vitesse,Limiteur de vitesse,CD/MP3/Bluetooth,Ordinateur de bord,Verrouillage centralisé à distance,Prix,Kilometrage
0,0,Temara,Fiat,Punto,2007,200000,249999,Diesel,5,Manuelle,...,False,True,False,False,False,True,False,False,60000,249999
1,1,Temara,Dacia,Dokker Van,2013,400000,449999,Diesel,6,Manuelle,...,False,False,False,False,False,False,False,False,70000,449999
2,2,Casablanca,Dacia,Dokker,2014,160000,169999,Diesel,6,Manuelle,...,False,False,False,False,False,False,False,False,90000,169999
3,3,Casablanca,Volkswagen,Touareg,2005,4999,0,Diesel,10,Automatique,...,False,False,False,False,False,False,False,False,90000,4999
4,4,Dakhla,Toyota,Prado,2007,200000,249999,Diesel,12,Manuelle,...,False,True,False,False,False,True,False,False,97000,249999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24771,24771,Marrakech,mini,cabrio,2011,70000,74999,Essence,9,Manuelle,...,False,True,True,False,False,False,False,False,180000,74999
24772,24772,Safi,mini,cooper,2007,190000,199999,Essence,9,Manuelle,...,False,True,True,False,False,True,False,False,63000,199999
24773,24773,Salé,mini,one,2006,10000,14999,Essence,8,Manuelle,...,False,True,False,False,False,True,False,False,60000,14999
24774,24774,Casablanca,mini,cooper,2010,140000,149999,Essence,9,Automatique,...,False,True,True,False,False,True,False,False,85000,149999


In [39]:
#sauvegarder le dataset:
df.to_csv('AvitoCarDataset_ready2Viz.csv', index=False)

<h2>Part 2: Data Exploration & Visualization</h2>

In [43]:
df = pd.read_csv("AvitoCarDataset_ready2Viz.csv")

def km_moy(df):
    #kilométrage moyen par marque
    kilometrage_moyen_par_marque = df.groupby('Marque')['Kilometrage'].mean().reset_index()
    source = ColumnDataSource(data=dict(Marque=kilometrage_moyen_par_marque['Marque'],Kilometrage=kilometrage_moyen_par_marque['Kilometrage']))
    p1 = figure(x_range=kilometrage_moyen_par_marque['Marque'], height=400, width=600, title="Kilométrage Moyen par Marque", toolbar_location=None, tools="")
    p1.vbar(x='Marque', top='Kilometrage', width=0.9, source=source, color='skyblue')
    #étiquettes des axes
    p1.xaxis.axis_label = "Marque"
    p1.yaxis.axis_label = "Kilométrage moyen"
    p1.xaxis.major_label_orientation = 1.0
    p1.xaxis.major_label_text_font_size = "10pt"
    #curseur interacgtif
    hover = HoverTool()
    hover.tooltips = [("Marque", "@Marque"), ("Kilométrage moyen", "@Kilometrage{0.0}")]
    p1.add_tools(hover)
    return p1

def prix_moy(df):
    # Calculer le prix moyen par marque
    prix_moyen_par_marque = df.groupby('Marque')['Prix'].mean().reset_index()
    # Créer une source de données ColumnDataSource
    source = ColumnDataSource(data=dict(Marque=prix_moyen_par_marque['Marque'], Prix=prix_moyen_par_marque['Prix']))
    # Créer la figure
    p2 = figure(x_range=prix_moyen_par_marque['Marque'], height=350, width=600, title="Chiffres de ventes moyen par Marque", toolbar_location=None, tools="")
    # Ajouter les barres verticales pour représenter le prix moyen par marque
    p2.vbar(x='Marque', top='Prix', width=0.9, source=source, color='orange')
    # Ajouter les étiquettes des axes
    p2.xaxis.axis_label = "Marque"
    p2.yaxis.axis_label = "Prix moyen"
    p2.xaxis.major_label_orientation = 1.0
    p2.xaxis.major_label_text_font_size = "10pt"
    # Ajouter des outils de survol interactifs pour afficher les détails
    hover = HoverTool()
    hover.tooltips = [("Marque", "@Marque"), ("Prix moyen", "@Prix{0.0}")]
    p2.add_tools(hover)
    # Afficher le graphique
    return p2


def marque_freq_ville(df):
    # Liste des villes principales
    villes_principales = ["Casablanca","Rabat","Marrakech","Fès","Tanger","Agadir","Salé","Meknès","Kénitra","Tétouan","Safi","Mohammedia","Khouribga","Béni Mellal","Nador"]
    # Filtrer le DataFrame pour inclure uniquement les villes principales
    df = df[df['Ville'].isin(villes_principales)]
    # Obtenir la marque la plus fréquente et son nombre d'occurrences pour chaque ville
    marque_plus_frequente_par_ville = df.groupby('Ville')['Marque'].agg(lambda x: x.value_counts().idxmax()).reset_index()
    occurrences_marque_plus_frequente = df.groupby('Ville')['Marque'].agg(lambda x: x.value_counts().max()).reset_index()
    # Créer la source de données ColumnDataSource
    source = ColumnDataSource(data=dict(Ville=marque_plus_frequente_par_ville['Ville'], Marque=marque_plus_frequente_par_ville['Marque'],Occurrences=occurrences_marque_plus_frequente['Marque']))
    # Créer la figure
    p3 = figure(y_range=villes_principales, height=350, width=650, title="Nombre d'occurrences de la marque la plus fréquente par ville", toolbar_location=None, tools="")
    # Créer les barres horizontales
    p3.hbar(y='Ville', right='Occurrences', height=0.9, source=source, color='skyblue')
    # Étiquettes des axes
    p3.xaxis.axis_label = "Nombre d'occurrences"
    p3.yaxis.axis_label = "Ville"
    # Outil de survol pour afficher les informations
    hover = HoverTool()
    hover.tooltips = [("Ville", "@Ville"), ("Marque", "@Marque"), ("Occurrences", "@Occurrences")]
    p3.add_tools(hover)
    # Afficher le graphique
    return p3

def occ_etats(df):
    # Calculer le nombre d'occurrences de chaque état groupé par la marque
    occurrence_etats = df.groupby(['Marque', 'État']).size().unstack(fill_value=0)
    # Créer une source de données ColumnDataSource
    source = ColumnDataSource(occurrence_etats)
    marques = occurrence_etats.index.tolist()
    # Créer la figure avec la liste des marques comme plage sur l'axe y
    p4 = figure(y_range=marques, height=400, width=750, title="Répartition des États par Marque",toolbar_location=None, tools="")
    # Ajouter les barres empilées pour chaque état
    p4.hbar_stack(stackers=['Bon', 'Très Bon', 'Excellent'], y='Marque', height=0.9, color=['lightblue', 'deepskyblue', 'dodgerblue'],source=source, legend_label=['Bon', 'Très Bon', 'Excellent'])
    # Ajouter les étiquettes des axes
    p4.yaxis.axis_label = "Marque"
    p4.xaxis.axis_label = "Fréquence"
    p4.yaxis.major_label_orientation = 0.0
    p4.yaxis.major_label_text_font_size = "10pt"
    # Ajouter des outils de survol interactifs pour afficher les détails
    hover = HoverTool()
    hover.tooltips = [("Marque", "@Marque"), ("Bon", "@Bon"), ("Très Bon", "@{Très Bon}"), ("Excellent", "@Excellent")]
    p4.add_tools(hover)
    # Afficher le graphique
    return p4


def carburant_by_marque(df):
    output_file("carburant_counts.html")
    # Préparation des données
    carburant_counts = df.groupby(['Marque', 'Type de carburant']).size().unstack().fillna(0)
    # Création du graphique
    brands = list(carburant_counts.index)
    carburants = list(carburant_counts.columns)
    data = { 'Marque': brands }
    data.update({ carburant: list(carburant_counts[carburant]) for carburant in carburants })
    source = ColumnDataSource(data=data)
    p5 = figure(x_range=brands, height=350, width=700, title="Répartition des types de carburant par marque", toolbar_location=None, tools="")
    renderers = p5.vbar_stack(carburants, x='Marque', width=0.9, color=Spectral6[:len(carburants)], source=source,legend_label=[str(x) for x in carburants])
    p5.y_range.start = 0
    p5.xgrid.grid_line_color = None
    p5.axis.minor_tick_line_color = None
    p5.outline_line_color = None
    p5.legend.location = "top_left"
    p5.legend.orientation = "horizontal"
    # Rotation et ajustement de la taille des étiquettes de l'axe des abscisses
    p5.xaxis.major_label_orientation = 1  # Rotation à 45 degrés
    p5.xaxis.major_label_text_font_size = "10pt"  # Taille de police des étiquettes
    hover = HoverTool(tooltips=[("Marque", "@Marque"),("Type de carburant", "$name"),("Valeur", "@$name")])
    p5.add_tools(hover)
    return p5



def heatmap_bv(df):
    output_file("heatmap_boite_vitess.html")
    top_cities = df['Ville'].value_counts().index[:20]  # Par exemple, les 20 villes avec le plus d'entrées
    filtered_data = df[df['Ville'].isin(top_cities)]
    # Préparation des données filtrées
    pivot = filtered_data.pivot_table(index='Ville', columns='Boite de vitesses', values='Prix', aggfunc='mean').fillna(0)
    pivot = pivot[['Automatique', 'Manuelle']]  # Inclure uniquement les colonnes valides
    pivot = pivot.stack().rename("Prix").reset_index()
    # Création du heatmap
    mapper = LinearColorMapper(palette=Viridis256, low=pivot['Prix'].min(), high=pivot['Prix'].max())
    p6 = figure(title="Heatmap de la boîte de vitesses par ville et prix (Top 20 villes)", x_range=['Automatique', 'Manuelle'], y_range=list(pivot['Ville'].unique()), x_axis_location="above", width = 600, height = 500, tools="hover,save,pan,box_zoom,reset")
    p6.rect(x="Boite de vitesses", y="Ville", width=1, height=1, source=ColumnDataSource(pivot), line_color=None, fill_color=transform('Prix', mapper))
    color_bar = ColorBar(color_mapper=mapper, location=(0, 0), ticker=BasicTicker())
    p6.add_layout(color_bar, 'right')
    # Améliorer la lisibilité des étiquettes
    p6.xaxis.major_label_orientation = 0  # Rotation des étiquettes de l'axe des x
    p6.yaxis.major_label_text_font_size = "10pt"  # Taille de police des étiquettes de l'axe y
    # Ajouter des info-bulles
    hover = HoverTool(tooltips=[("Ville", "@Ville"),("Boîte de vitesses", "@{Boite de vitesses}"),("Prix moyen", "@Prix{0,0}")])
    p6.add_tools(hover)
    return p6

def security_heatmap(df):
    # Préparation des données
    securite_columns = ['ABS', 'ESP', 'Airbags']
    securite_counts = df.groupby(['Marque'])[securite_columns].sum()
    # Transformation des données pour Bokeh
    securite_counts = securite_counts.stack().rename("count").reset_index()
    securite_counts.columns = ['Marque', 'Système de sécurité', 'count']
    # Création du heatmap
    mapper = LinearColorMapper(palette=Viridis256, low=securite_counts['count'].min(), high=securite_counts['count'].max())
    p7 = figure(title="Heatmap des systèmes de sécurité par marque", x_range=securite_columns, y_range=list(securite_counts['Marque'].unique()), x_axis_location="above", width = 600, height = 500, tools="hover,save,pan,box_zoom,reset")
    p7.rect(x="Système de sécurité", y="Marque", width=1, height=1, source=ColumnDataSource(securite_counts),line_color=None, fill_color=transform('count', mapper))
    color_bar = ColorBar(color_mapper=mapper, location=(0, 0), ticker=BasicTicker())
    p7.add_layout(color_bar, 'right')
    # Ajouter des info-bulles
    hover = HoverTool(tooltips=[("Marque", "@Marque"),("Système de sécurité", "@{Système de sécurité}"),("Count", "@count")])
    p7.add_tools(hover)
    return p7

def scatter_plot(df):
    source = ColumnDataSource(df)
    p8 = figure(title="Puissance fiscale par année modèle", width=600, height=400, x_axis_label='Année-Modèle', y_axis_label='Puissance fiscale', tools="pan,wheel_zoom,box_zoom,reset")
    p8.scatter(x='Année-Modèle', y='Puissance fiscale', source=source, size=7, color="navy", alpha=0.5)
    return p8




kilom_Moy = km_moy(df)
car_mean_price = prix_moy(df)
villeMarque_freq = marque_freq_ville(df)
car_etats_occ = occ_etats(df)
carb_marque = carburant_by_marque(df)
bv_heatmap = heatmap_bv(df)
sec_heatmap = security_heatmap(df)
scatter_plot_p = scatter_plot(df)


In [44]:
show(kilom_Moy)

In [45]:
show(car_mean_price)

In [46]:
show(villeMarque_freq)

In [47]:
show(car_etats_occ)

In [48]:
show(carb_marque)

In [49]:
show(bv_heatmap)

In [50]:
show(sec_heatmap)

In [51]:
show(scatter_plot_p)